<a href="https://colab.research.google.com/github/SirwaniViren/MSc-Gousto-Project/blob/main/initial_framework_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import math
import numpy as np
import pandas as pd
import random
from collections import defaultdict

In [10]:
random.seed(42)

In [11]:
# Dataframe of orders
# How many recipes per box - between 1 and 5
# lead day 5
orders_to_recipe_ld5_df = pd.DataFrame({'order_id': [1, 1, 1, 2, 2, 3, 3, 4, 4],
                                        'item_id': [10, 20, 40, 10, 30, 20, 50, 20, 40],
                                        'leadday': [5, 5, 5, 5, 5, 5, 5, 5, 5],
                                        'site': ['F1', 'F1', 'F1', 'F2', 'F2', 'F2', 'F2', 'F3', 'F3']})

# lead day 0
orders_to_recipe_ld0_df = pd.DataFrame({'order_id': [1, 1, 1, 2, 2, 3, 3, 4, 4],
                                        'item_id': [10, 20, 40, 10, 30, 20, 50, 20, 30],
                                        'leadday': [0, 0, 0, 0, 0, 0, 0, 0, 0],
                                        'site': ['F1', 'F1', 'F1', 'F2', 'F2', 'F3', 'F3', 'F2', 'F2']})

# Set of recipe per factory
eligibility_id = {'F1': [10, 20, 40], 'F2': [10, 20, 30, 50], 'F3': [10, 20, 30, 40, 50]}

# Should be computed from a set of recipes per factory
# Ideally, elgibility for 3 factories will go from F1: 30%, F2, 60% and F3: 100%
# eligible boxes per factory
orders_eligibility_df = pd.DataFrame({'order_id': [1, 2, 3, 4], 'eligibility': [['F1', 'F3'], ['F2', 'F3'], ['F2', 'F3'], ['F3', 'F2']]})

# Caps:
factory_caps = {'F1': 1, 'F2': 2, 'F3': np.float32('inf')}



In [12]:
# Allocate at LD5
# Greedy algorithm


# Allocate at LD0
# Greedy algorithm


# Output
# item_id site lead_5 lead_0  abs_error
# 1        F1     4     3

# Compute Global error and site specific error
# In theory should have wmape_global <= wmape_site

# Can you minimise the site specific error within the constraints - only allowed to modified my allocation at LD0
# swap? heuristic? meta-heuristic such GA, simulated annealing




In [17]:
def allocate_orders(orders_df, eligibility_df, factory_caps, eligibility_id):
    # Initialize the allocation dictionary with default values of 0
    allocation = defaultdict(lambda: defaultdict(int))

    # Iterate over each order in the orders DataFrame
    for _, order in orders_df.iterrows():
        order_id = order['order_id']
        item_id = order['item_id']

        # Get the eligible factories for the current order_id
        eligible_factories = eligibility_df.loc[eligibility_df['order_id'] == order_id, 'eligibility'].values[0]

        # Try to allocate the item to an eligible factory
        for factory in eligible_factories:
            if item_id in eligibility_id[factory] and factory_caps[factory] > 0:
                allocation[item_id][factory] += 1
                factory_caps[factory] -= 1
                break

    return allocation

# Allocate orders for LD5 and LD0
ld5_allocation = allocate_orders(orders_to_recipe_ld5_df, orders_eligibility_df, factory_caps.copy(), eligibility_id)
ld0_allocation = allocate_orders(orders_to_recipe_ld0_df, orders_eligibility_df, factory_caps.copy(), eligibility_id)


In [21]:
# Prepare the result dataframe
data = []
for item_id in set(orders_to_recipe_ld5_df['item_id']).union(set(orders_to_recipe_ld0_df['item_id'])):
    for factory in eligibility_id.keys():
        lead_5 = ld5_allocation[item_id][factory] if item_id in ld5_allocation else 0
        lead_0 = ld0_allocation[item_id][factory] if item_id in ld0_allocation else 0
        abs_error = abs(lead_5 - lead_0)
        data.append([item_id, factory, lead_5, lead_0, abs_error])

result_df = pd.DataFrame(data, columns=['item_id', 'site', 'lead_5', 'lead_0', 'abs_error'])
print(result_df)

    item_id site  lead_5  lead_0  abs_error
0        40   F1       0       0          0
1        40   F2       0       0          0
2        40   F3       2       1          1
3        10   F1       1       1          0
4        10   F2       1       1          0
5        10   F3       0       0          0
6        50   F1       0       0          0
7        50   F2       0       0          0
8        50   F3       1       1          0
9        20   F1       0       0          0
10       20   F2       0       0          0
11       20   F3       3       3          0
12       30   F1       0       0          0
13       30   F2       1       1          0
14       30   F3       0       1          1


In [ ]:
factory_capacities = {}

# Loop through factories 1 to 99 and assign random capacities
for i in range(1, 100):
    factory_capacities[f"Factory_{i}"] = random.choice(range(1000, 10001, 1000))

# Factory 100 has infinite capacity
factory_capacities["Factory_100"] = float('inf')

In [ ]:
print(list(factory_capacities.items())[0])

('Factory_1', 3000)


In [ ]:
# Sample data
recipes = [f"Recipe_{i}" for i in range(1, 501)]
factories = [f"Factory_{i}" for i in range(1, 101)]

# Create a dictionary for factories and the recipes they offer
factory_recipes = {factory: np.random.choice(recipes, size=np.random.randint(1, 50), replace=False).tolist() for factory in factories}

In [ ]:
# Initialize data for day 18
data = []
for factory, recs in factory_recipes.items():
    for recipe in recs:
        forecast = np.random.randint(1, 100)  # Simulated forecast data
        hard_allocation = np.random.randint(1, 100)  # Simulated hard allocation data
        abs_error = abs(forecast - hard_allocation)
        data.append([recipe, factory, forecast, hard_allocation, abs_error])

# Create DataFrame
df = pd.DataFrame(data, columns=["Recipe", "Factory", "Forecast", "Hard_Allocation", "Absolute_Error"])


In [ ]:
# Display the DataFrame
print(df.head())

       Recipe    Factory  Forecast  Hard_Allocation  Absolute_Error
0  Recipe_490  Factory_1        62               89              27
1   Recipe_62  Factory_1        25               23               2
2  Recipe_253  Factory_1        61               36              25
3   Recipe_20  Factory_1        26                5              21
4  Recipe_384  Factory_1        71               32              39


In [ ]:
# Calculate WMAPE
total_forecast = df["Forecast"].sum()
wmape = df["Absolute_Error"].sum() / total_forecast

# Display WMAPE
print(f"Weighted Mean Absolute Percentage Error (WMAPE): {wmape:.4f}")

Weighted Mean Absolute Percentage Error (WMAPE): 0.6561
